# 프로젝트 Movielnes 영화 추천실습
   
     
        
        

``` wget http://files.grouplens.org/datasets/movielens/ml-1m.zip```

## libaryry import

In [1]:
import os
import pandas as pd 
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

## data improt


In [2]:
#rating_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/ratings.dat'
rating_file_path= './data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## data preprocessing

In [3]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)


In [5]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [6]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'movie_id', 'count']
data = ratings[using_cols]
data.tail(10)

,user_id,movie_id,count
1000198,6040,2021,3
1000199,6040,2022,5
1000200,6040,2028,5
1000201,6040,1080,4
1000202,6040,1089,4
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
movie_file_path= './data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.tail(10)

,movie_id,title,genre
3873,3943,Bamboozled (2000),Comedy
3874,3944,Bootmen (2000),Comedy|Drama
3875,3945,Digimon: The Movie (2000),Adventure|Animation|Children's
3876,3946,Get Carter (2000),Action|Drama|Thriller
3877,3947,Get Carter (1971),Thriller
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [8]:
# 유저 이름을 보기 위해 메타 데이터를 읽어옵니다.
#movie_file_path=os.getenv('HOME') + '/aiffel/AIFFEL_LSG/lms/E07-Recommender_sys/data/ml-1m/movies.dat'
users_file_path= './data/ml-1m/users.dat'
users_cols = ['user_id', 'gender', 'age','occupation','Zip-code' ] 
users = pd.read_csv(users_file_path, sep='::', names=users_cols, engine='python')
users.tail(10)

,user_id,gender,age,occupation,Zip-code
6030,6031,F,18,0,45123
6031,6032,M,45,7,55108
6032,6033,M,50,13,78232
6033,6034,M,25,14,94117
6034,6035,F,25,1,78734
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060
6039,6040,M,25,6,11106


In [9]:
#user_unique = data['user_id'].unique()
movie_names = movies[['movie_id','title']]
movie_names

#movie_title = {val:idx for idx, val in movies['movie_id'].data,movies['title'].data}
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
#user_to_idx = {v:k for k,v in users.items()}
#movie_to_idx = {v:k for k,v in  enumerate(artist_unique)}

,movie_id,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
3878,3948,Meet the Parents (2000)
3879,3949,Requiem for a Dream (2000)
3880,3950,Tigerland (2000)
3881,3951,Two Family House (2000)


In [10]:
ratings.nunique()

user_id        6039
movie_id       3628
count             3
timestamp    412911
dtype: int64

## data <분석
- ratings에 있는 유니크한 영화 개수
- rating에 있는 유니크한 사용자 수
- 가장 인기있는 영화 30개(인기순)

In [11]:
# 영화 개수
data['movie_id'].nunique()

3628

In [12]:
# 사용자 수
data['user_id'].nunique()

6039

In [13]:
# 가장 인기 있는 영화 30개 인기수
#TODO movie id 와 movies title 매핑
movie_count= data.groupby('movie_id')['user_id'].count()
#movie_count['movie_id'].map(movies.get).dropna()
#print(movies[movies['movie_id'] == 2858]['title'])   movies[movie_count['movie_id'] == movies['movie_id']]['title']
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [14]:
print(movies[movies['movie_id'] == 2858]['title']) 

2789    American Beauty (1999)
Name: title, dtype: object


## data add 
- 내가 선호하는 영화 5가지 골라서 rating 추가


In [15]:
movies[movies["genre"]== "Action"].head(10)
#movies[movies["genre"]== "Drama"].head(30)

,movie_id,title,genre
8,9,Sudden Death (1995),Action
19,20,Money Train (1995),Action
70,71,Fair Game (1995),Action
143,145,Bad Boys (1995),Action
202,204,Under Siege 2: Dark Territory (1995),Action
224,227,Drop Zone (1994),Action
248,251,"Hunted, The (1995)",Action
312,315,"Specialist, The (1994)",Action
380,384,Bad Company (1995),Action
389,393,Street Fighter (1994),Action


In [16]:

movies[movies['title']=='Bad Company (1995)']['movie_id']


380    384
Name: movie_id, dtype: int64

In [17]:
my_favorite = ['Bad Company (1995)' , 
               'Street Fighter (1994)' ,
               'Bad Boys (1995)' ,
               'Sudden Death (1995)' ,
               'Fair Game (1995)']
my_favorite_id= [384, 393, 145, 9, 71 ]


In [18]:
data

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [19]:
ratings

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [20]:
 


my_playlist = pd.DataFrame({'user_id': [6041]*5, 
                            'movie_id':my_favorite_id, 
                            'count':4})

if not data.isin({'user_id':['6041']})['user_id'].any(): 
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
#data = data.append(my_playlist)     
data.tail(10)

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,384,4
1,6041,393,4
2,6041,145,4
3,6041,9,4
4,6041,71,4


In [21]:
data['user_id'].nunique()
# 1명 추가 됨...

6040

In [22]:
data

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
0,6041,384,4
1,6041,393,4
2,6041,145,4
3,6041,9,4


## csr matrix 

In [29]:
#from scipy.sparse import csr_matrix



num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()
print(num_user, num_movie)
# print(data2['count'].shape)

# print(type(data))
# print(type(num_user))
# print(type(data2.user_id))
# print(type(data2['count']))

#csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (num_user, num_movie)) 
csr_data = csr_matrix((data['count'], (data.user_id, data.movie_id)), shape= (6050, 4000)) 
csr_data

6040 3628


<6050x4000 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [ ]:
data.nunique()

In [ ]:
# num_user = ratings['user_id'].nunique()
# num_movie = ratings['movie_id'].nunique()
# print(num_user, num_movie)
# print(ratings['count'].shape)

# csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie_id'])), shape= (num_user,num_movie))
# csr_data

## als_model = AlternatingLeastSquares 모델 훈련

In [30]:
# Implicit AlternatingLeastSquares 모델의 선언
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [31]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<4000x6050 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [32]:
# 모델 훈련
als_model.fit(csr_data_transpose)

## 선호도 파악

In [35]:
# 나의 선호도
me_idx = 6041
me_vector  = als_model.user_factors[me_idx]
print(me_vector)


[ 0.3220228  -0.26612803  0.07191087  0.15949483 -0.09219621  0.41332713
  0.06473979  0.05634092 -0.30008054  0.20807798 -0.20641813 -0.16052487
 -0.02705514  0.15539066  0.22086035  0.03658444  0.24402499  0.22372353
 -0.2666574  -0.13084082  0.24677101 -0.29619813 -0.12389661 -0.12441921
  0.10542396  0.07729556 -0.41072756 -0.13440621  0.05892525 -0.11598359
  0.31264418  0.06389473  0.30311608 -0.37629074 -0.33920196 -0.1615162
 -0.26244998  0.2839672  -0.12159638 -0.07983002  0.09412021  0.15323326
 -0.35352623 -0.21745276  0.32966885  0.17249547 -0.00543889  0.19145271
  0.03178566  0.19071878 -0.15998866 -0.37387902 -0.00491816 -0.1044338
 -0.11918101  0.27108264  0.03236694 -0.5941094  -0.35654572  0.16059326
  0.16266061  0.14141434 -0.1523178   0.22002539  0.00374068 -0.23647812
  0.29686445  0.30855414 -0.28220674  0.20418732  0.51323336  0.01333146
 -0.13638097  0.14408137  0.23368225 -0.11923807  0.07983219  0.05162042
 -0.33491835 -0.29575056  0.12763172  0.06410906  0.2

In [37]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
fg_vector = als_model.item_factors[fg_idx]
print(fg_vector)

np.dot(me_vector, fg_vector)

[ 1.10967038e-02 -2.96933926e-03  5.62351337e-03  9.88563616e-03
  1.09446747e-03  6.73755771e-03 -2.15929365e-04 -1.45664264e-03
 -1.77117938e-03  1.19495187e-02 -2.72778189e-03  2.24393676e-03
 -1.80894148e-03  7.97402114e-03  7.91889895e-03  1.00568042e-03
  6.86946604e-03  8.33794009e-03 -5.42217586e-03 -3.06127639e-03
  1.09088225e-02  7.87901867e-04  5.25907660e-03 -1.37929048e-03
  5.07648522e-03  4.91851615e-03 -2.57210853e-03  4.68331343e-03
  1.65439269e-03  1.93637586e-03  1.00474618e-02  5.38314180e-03
  2.84701400e-03 -1.04636117e-03  4.11276938e-03  1.13286439e-03
  1.33302121e-03  1.25469258e-02 -1.71427440e-03  8.36250474e-05
  2.90138624e-03  1.18717169e-02 -1.70056056e-03  1.10130722e-03
  7.01135350e-03  5.78091806e-03  3.72962514e-03  7.17710098e-03
  1.08278086e-02  5.86572615e-03  2.23874347e-03 -3.01953708e-03
  9.79351695e-04 -1.87012472e-03 -3.32462974e-03  6.27356209e-03
  5.14742732e-03  7.01867917e-04  2.34522391e-03  4.76170518e-03
  7.23972917e-03  8.52773

0.071545124

In [40]:
#'Street Fighter (1994)' , idx 393 선호도   movie_names
sf_idx = 393
sf_vector = als_model.item_factors[sf_idx]
print(sf_vector)

np.dot(me_vector, sf_vector)


[ 7.1404469e-03 -1.8514909e-03  3.1845197e-03  5.8155260e-03
  2.5637101e-03  1.9645861e-03 -1.6489642e-03  7.0024241e-04
 -1.0038090e-03  5.5202609e-03 -2.1261778e-03 -2.2230605e-03
  8.1436512e-05  6.9407243e-03  5.8848271e-03  6.5830420e-04
  3.2552583e-03  9.4572245e-04 -2.6366294e-03  1.6843490e-03
  5.4282052e-03  7.9677161e-04 -3.5719073e-03  5.4745113e-05
 -2.5780137e-05  2.1213018e-03 -3.1978777e-03 -5.7916943e-04
 -1.9820307e-03  5.1719793e-03  7.5249230e-03  3.6770247e-03
  3.6217310e-03 -5.8228467e-03  3.0694650e-03 -4.5495181e-04
 -1.7450574e-04  1.0625007e-02  3.8173667e-03  2.7127233e-03
  3.8133212e-03  1.3177133e-03 -2.1563331e-03 -5.9300619e-06
  7.6320223e-03  4.0836044e-04  6.1026956e-03  7.0553338e-03
  4.9664481e-03  5.7333703e-03  2.7544121e-04 -4.9191830e-03
  7.9983077e-04 -3.8120595e-03 -5.7568913e-03  8.3740428e-03
  6.0220221e-03 -2.4785956e-03  1.5628955e-03  8.8693202e-03
  6.8941168e-03 -3.8835959e-05 -3.5001792e-03  6.0208078e-04
  3.0726413e-03  2.51069

0.056128547

In [42]:
#'Bad Boys' , idx 393 선호도   movie_names
bb_idx = 145
bb_vector = als_model.item_factors[bb_idx]
print(bb_vector)

np.dot(me_vector, bb_vector)
# my_favorite = ['Bad Company (1995)' , 
#                'Street Fighter (1994)' ,
#                'Bad Boys' ,
#                'Sudden Death (1995)' ,
#                'Fair Game (1995)']
# my_favorite_id= [384, 393, 145, 9, 71 ]

[ 0.01235456 -0.00631395  0.00552482  0.00624047  0.00774966  0.02798221
  0.01240891  0.00401246  0.00116683  0.01374004 -0.00743233 -0.01054189
  0.00487122 -0.00261119  0.01188714  0.01998491  0.01424464  0.01958132
 -0.00112759 -0.00581937  0.00119223  0.0006045  -0.0041312   0.00442379
 -0.00262082  0.00201938 -0.01815773  0.00080731  0.01995449 -0.01087386
  0.01267361  0.0075448   0.0166564  -0.00973254 -0.03488528  0.00778198
 -0.01446448  0.01231759  0.00139378  0.00470806  0.00265989  0.01255405
 -0.00456472 -0.0064685   0.01682308  0.01834571 -0.00259483  0.00159539
 -0.00872884  0.00530275  0.00543365  0.00190804  0.00801707  0.01457253
  0.00845643  0.02104814  0.00376734 -0.02790561 -0.01142661  0.00587012
  0.00988239  0.01484061  0.00555872  0.01493063 -0.0009696  -0.00778786
  0.00891969  0.02187785 -0.01332749  0.01443464  0.02238261  0.00926501
 -0.00099861  0.01823101  0.01172351  0.00954167  0.00797935  0.00420042
 -0.00192685 -0.00610813  0.01964442  0.00528354  0

0.1846596

## 유사 영화 추천

In [46]:
#'Fair Game (1995)'  idx 71 선호도   movie_names
fg_idx = 71
similar_movies = als_model.similar_items(fg_idx, N=15)
similar_movies

[(71, 0.053732514),
 (1004, 0.04633327),
 (544, 0.045934606),
 (1626, 0.04544114),
 (1744, 0.04516598),
 (548, 0.044705037),
 (694, 0.044382308),
 (1604, 0.044320963),
 (20, 0.04430308),
 (1497, 0.04424421),
 (990, 0.04389384),
 (2835, 0.043833237),
 (9, 0.043556042),
 (227, 0.043470196),
 (1869, 0.043250542)]

In [71]:
# title 
for m in similar_movies:
    print(movies[movies['movie_id']==m[0]][['title','genre']])

               title   genre
70  Fair Game (1995)  Action
                       title            genre
991  Glimmer Man, The (1996)  Action|Thriller
                        title   genre
540  Striking Distance (1993)  Action
                       title                  genre
1583  Fire Down Below (1997)  Action|Drama|Thriller
                 title                      genre
1693  Firestorm (1998)  Action|Adventure|Thriller
                        title   genre
544  Terminal Velocity (1994)  Action
                      title   genre
685  Substitute, The (1996)  Action
                   title          genre
1563  Money Talks (1997)  Action|Comedy
                 title   genre
19  Money Train (1995)  Action
                   title   genre
1465  Double Team (1997)  Action
                   title                      genre
978  Maximum Risk (1996)  Action|Adventure|Thriller
                    title            genre
2766  Chill Factor (1999)  Action|Thriller
                 title  

## 좋아할 만한 영화 추천

In [58]:
# zimin에게 
me_idx = 6041
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(me_idx, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(204, 0.10406698),
 (44, 0.09517281),
 (464, 0.094421),
 (423, 0.09407365),
 (20, 0.09317492),
 (1385, 0.09145656),
 (434, 0.090557516),
 (2808, 0.09055193),
 (886, 0.08906894),
 (227, 0.08563958),
 (1866, 0.084755726),
 (1004, 0.08411434),
 (548, 0.08318102),
 (694, 0.08281553),
 (511, 0.08239934),
 (1687, 0.08123389),
 (315, 0.080340624),
 (1203, 0.07976359),
 (479, 0.07932318),
 (1382, 0.07912278)]

In [69]:
## 변환
a = []
# title 
for m in movie_recommended:
    a.append(movies[movies['movie_id']==m[0]]['title'])
    print(movies[movies['movie_id']==m[0]][['title','genre']])

                                    title   genre
202  Under Siege 2: Dark Territory (1995)  Action
                   title             genre
43  Mortal Kombat (1995)  Action|Adventure
                  title                            genre
460  Hard Target (1993)  Action|Adventure|Crime|Thriller
                 title            genre
419  Blown Away (1994)  Action|Thriller
                 title   genre
19  Money Train (1995)  Action
                   title   genre
1364  Under Siege (1992)  Action
                  title                   genre
430  Cliffhanger (1993)  Action|Adventure|Crime
                         title          genre
2739  Universal Soldier (1992)  Action|Sci-Fi
                  title   genre
874  Bulletproof (1996)  Action
                title   genre
224  Drop Zone (1994)  Action
                    title          genre
1797  Big Hit, The (1998)  Action|Comedy
                       title            genre
991  Glimmer Man, The (1996)  Action|Thriller
      

## 코멘트....
- 주입한 데이터를 액션영화 5개만을 골라서 입력해서 그런지 유사 영화와 추천영화의 장르가 전부 액션으로 나왔다.